# authors: Rambod Azimi & Mahdi Moghassemi
# Date: May 2023

In [ ]:
# importing the necessary libraries to the project
import numpy as np
import math, copy
import pandas as pd
from sklearn.linear_model import SGDRegressor # linear regression model library in scikit-learn
from sklearn.preprocessing import StandardScaler # Z-score scaling library in scikit-learn

In [ ]:
# This method will replace non-numeric values in the dataset to numeric and computable values
def replace_data(initial_data):
    #  yes --> 1
    # no --> 0

    initial_data['mainroad'] = initial_data['mainroad'].replace(['yes'], 1)
    initial_data['mainroad'] = initial_data['mainroad'].replace(['no'], 0)
    
    initial_data['guestroom'] = initial_data['guestroom'].replace(['yes'], 1)
    initial_data['guestroom'] = initial_data['guestroom'].replace(['no'], 0)

    initial_data['basement'] = initial_data['basement'].replace(['yes'], 1)
    initial_data['basement'] = initial_data['basement'].replace(['no'], 0)

    initial_data['hotwaterheating'] = initial_data['hotwaterheating'].replace(['yes'], 1)
    initial_data['hotwaterheating'] = initial_data['hotwaterheating'].replace(['no'], 0)

    initial_data['airconditioning'] = initial_data['airconditioning'].replace(['yes'], 1)
    initial_data['airconditioning'] = initial_data['airconditioning'].replace(['no'], 0)

    initial_data['prefarea'] = initial_data['prefarea'].replace(['yes'], 1)
    initial_data['prefarea'] = initial_data['prefarea'].replace(['no'], 0)

    initial_data['furnishingstatus'] = initial_data['furnishingstatus'].replace(['furnished'], 2) 
    initial_data['furnishingstatus'] = initial_data['furnishingstatus'].replace(['semi-furnished'], 1)
    initial_data['furnishingstatus'] = initial_data['furnishingstatus'].replace(['unfurnished'], 0)

    return initial_data

In [ ]:
def predict(x, w, b): 
    """
    prediction of all the training examples using linear regression
    Args:
      x (ndarray): Shape (n,) example with multiple features
      w (ndarray): Shape (n,) model parameters   
      b (scalar):             model parameter 
      
    Returns:
      p (scalar):  prediction
    """
    m = x.shape[0] # number of training examples (545)
    f = np.zeros(m)
    for i in range(m):
      f[i] = np.dot(x[i,:], w[i,:]) + b
    return f # f[i] = x1w1 + x2w2 + ... + b  

In [ ]:
initial_data = pd.read_csv('Housing.csv') # reading the dataset from a csv file in the same directory
y_train = initial_data.iloc[:, 0].to_numpy() # target values (1D array)
initial_data = replace_data(initial_data) # make all the feature variables numeric using this helper function
x_train = initial_data.iloc[:, 1:].to_numpy() # feature values (2D array) --> skip the first row

In [ ]:
# just for testing purposes
# data is stored in numpy array/matrix
print(f"x Shape: {x_train.shape}, X Type:{type(x_train)})")
print(x_train)
print(f"y Shape: {y_train.shape}, y Type:{type(y_train)})")
print(y_train)

In [ ]:
# cost function J(w, b) to check the accuracy of the generated model
def compute_cost(x, y, w, b):
    m = x.shape[0]
    total_cost = 0.0

    for i in range(m):
        f = np.dot(x[i], w) + b
        total_cost += (f - y[i]) ** 2
    total_cost /= (2 * m)

    return total_cost

In [ ]:
# testing the cost function
cost = compute_cost(x_train, y_train, x_train[0], 0.0)
print(f"Cost at optimal w: {cost}")

In [ ]:
def compute_gradient(x, y, w, b):
    m = x.shape[0]
    n = x.shape[1]

    dj_dw = np.zeros((n,))
    dj_db = 0

    for i in range(m):
        f = np.dot(x[i], w) + b
        error = f - y[i]

        for j in range(n):
            dj_dw[j] += error * x[i, j]
        dj_db += error
    dj_dw /= m
    dj_db /= m

    return dj_db, dj_dw

In [ ]:
# testying compute_gradient method
temp_dj_dw, temp_dj_db = compute_gradient(x_train, y_train, x_train[0], 0.0)
print(f"dj_db at initial w, b: {temp_dj_db}")
print(f"dj_dw at initial w, b: {temp_dj_dw}")

In [ ]:
def gradient_descent(x, y, w_in, b_in, alpha, iterations):
    w = copy.deepcopy(w_in)
    b = b_in

    for i in range(iterations):
        dj_db, dj_dw = compute_gradient(x, y, w, b)

        w = w - alpha * dj_dw
        b - b - alpha * dj_db

    return w, b

In [ ]:
initial_w = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 10, 20, 10, 0, 20, 0])
initial_b = 2.0
scaler = StandardScaler()
x_normalized = scaler.fit_transform(x_train) # normalized (Z-score) version of x_train 

iterations = 1900
alpha = 5.0e-7

w_final, b_final = gradient_descent(x_normalized, y_train, initial_w, initial_b, alpha, iterations)

total_accuracy = 0.0

for i in range(x_train.shape[0]):
    accuracy = 0.0
    if(((np.dot(x_train[i], w_final) + b_final) / y_train[i]) * 100 > 100):
        accuracy = (y_train[i] / (np.dot(x_train[i], w_final) + b_final)) * 100
    else:
        accuracy = ((np.dot(x_train[i], w_final) + b_final) / y_train[i]) * 100
    print(f"prediction: {np.dot(x_train[i], w_final) + b_final:0.0f}, target value: {y_train[i]}, accuracy: {accuracy:0.2f}%")
    total_accuracy += accuracy

print(f"Overal accuracy: {total_accuracy / x_train.shape[0]:0.2f}%")

In [ ]:
# input from the user to predit the price
while(True):
    area = input("Please enter the area (Square foot): ")
    bedroom = input("Please enter the number if bedrooms: ")
    bathroom = input("Please enter the number of bathrooms: ")
    story = input("Please enter the number of stories: ")

    mainroad = input("Is the house on the main road (y/n)? ")
    if (mainroad == 'y'):
        mainroad2 = "yes"
        mainroad = 1
    elif (mainroad == 'n'):
        mainroad = 0
        mainroad2 = "no"
    else:
        print("wrong answer in mainroad!")

    guestroom = input("Does the house have any guest room (y/n)? ")
    if (guestroom == 'y'):
        guestroom = 1
        guestroom2 = "yes"
    elif (guestroom == 'n'):
        guestroom = 0
        guestroom2 = "no"
    else:
        print("wrong answer in guest room!")

    basement = input("Does the house have basement (y/n)? ")
    if (basement == 'y'):
        basement = 1
        basement2 = "yes"
    elif (basement == 'n'):
        basement = 0
        basement2 = "no"
    else:
        print("wrong answer in basement!")

    hot_water_heating = input("Does the house have heating system (y/n)? ")
    if (hot_water_heating == 'y'):
        hot_water_heating = 1
        hot_water_heating2 = "yes"
    elif (hot_water_heating == 'n'):
        hot_water_heating = 0
        hot_water_heating2 = "no"
    else:
        print("wrong answer in heating!")

    air_conditioning = input("Does the house have air conditioning (y/n)? ")
    if (air_conditioning == 'y'):
        air_conditioning = 1
        air_conditioning2 = "yes"
    elif (air_conditioning == 'n'):
        air_conditioning = 0
        air_conditioning2 = "no"
    else:
        print("wrong answer in air conditioning!")

    parking = input("Please enter the number of parkings: ")
    prefarea = input("Does the house have prefarea (y/n)? ")
    if (prefarea == 'y'):
        prefarea = 1
        prefarea2 = "yes"
    elif (prefarea == 'n'):
        prefarea = 0
        prefarea2 = "no"
    else:
        print("wrong answer in air prefarea!")

    furnished = input("Is the house furnished (y), semi-furnished (s), or unfurnished (n)? ")
    if (furnished == 'y'):
        furnished = 2
        furnished2 = "furnished"
    elif (furnished == 's'):
        furnished = 1
        furnished2 = "semi-furnished"
    elif (furnished == 'n'):
        furnished = 0
        furnished2 = "unfurnished"
    else:
        print("wrong answer in air furnished!")

    print(f"You entered the following values: \n area: {area} \n bedrooms: {bedroom} \n bathrooms: {bathroom} \n stories: {story} \n mainroad: {mainroad2} \n guestroom: {guestroom2} \n basement: {basement2} \n heating system: {hot_water_heating2} \n airconditioning: {air_conditioning2} \n parking: {parking} \n prefarea: {prefarea2} \n furnishing status: {furnished2}")
    x_train_input = np.array([int(area), int(bedroom), int(bathroom), int(story), int(mainroad), int(guestroom), int(basement), int(hot_water_heating), int(air_conditioning), int(parking), int(prefarea), int(furnished)])

    result = np.dot(w_final, x_train_input) + b_final
    print(f"Prediction: {result:0.2f}")

    break